In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import warnings
import pandas as pd
import time
import os
from tqdm.notebook import tqdm
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
from collections import Counter
from collections import defaultdict
#from scipy.sparse import csr_matrix
import numpy as np
#from sklearn.preprocessing import normalize
import math
#from konlpy.tag import Twitter
from nltk.tag import pos_tag
from nltk.tag import untag
from nltk import Text
#from matplotlib import pyplot as plt
from nltk import FreqDist
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer, sent_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer, WordNetLemmatizer
from glob import glob
from tqdm.notebook import tqdm
import collections
import os.path
from googletrans import Translator
from itertools import chain
from collections import defaultdict


def first_collect_url():
    
    """다음 페이지로 이동하는 경로 수집"""
    
    first_to_second_url_list = [] 
    cla1_cla2_dict = collections.defaultdict(list)
    cla1_seeall_url_dict = collections.defaultdict(list)
    third_url_list = []
    see_all_url = []
    
    driver.get('https://paperswithcode.com/sota')
    fs_url = driver.find_elements(By.CLASS_NAME,'col-md-12 > h4 > a')
    
    for url in fs_url:
        first_to_second_url_list.append(url.get_attribute('href'))

    for url in first_to_second_url_list:
        driver.get(url)

        cla1_elem = driver.find_elements(By.CLASS_NAME, 'col-lg-12 > h1')
        cla2_elem = driver.find_elements(By.CLASS_NAME, 'col-xl-8.card-col.card-col-title > h1')
        seeall_url_elem = driver.find_elements(By.CLASS_NAME, 'sota-all-tasks > a')
        third_url_elem = driver.find_elements(By.CLASS_NAME,'card > a')

        for cla1 in cla1_elem:
            for cla2 in cla2_elem:
                cla1_cla2_dict[cla1.text].append(cla2.text)
            for seeall_url in seeall_url_elem:
                cla1_seeall_url_dict[cla1.text].append(seeall_url.get_attribute('href'))

        for th_url in third_url_elem:
            if th_url.get_attribute('href') not in third_url_list:
                third_url_list.append(th_url.get_attribute('href'))


def make_classification():
    
    """cla1_cla2_dict.csv 생성
    cla1, cla2는 분류기준이며 cla2는 cla1에 속해있다"""
    
    for cla1, urls in cla1_seeall_url_dict.items():
        for url in urls:
            driver.get(url)
            cla2_elem = driver.find_elements(By.CLASS_NAME, 'card-title')

            for cla2 in cla2_elem:
                if cla2.text not in cla1_cla2_dict[cla1]:
                    cla1_cla2_dict[cla1].append(cla2.text)
                    
    for url in see_all_url:
        driver.get(url)
        third_url_elem2 = driver.find_elements(By.CLASS_NAME,'card > a')

    for th_url in third_url_elem2:
        if th_url.get_attribute('href') not in third_url_list:
            third_url_list.append(th_url.get_attribute('href'))
            
    cla1_cla2_df = pd.DataFrame.from_dict(cla1_cla2_dict, orient='index')
    cla1_cla2_df.to_csv('cla1_cla2_dict.csv')
            
    third_url_df = pd.DataFrame(third_url_list, columns=['third_url'])
    third_url_df.to_csv('third_url.csv')


def collect_title_url():
    
    """논문의 title, url 수집, title_url.csv, cla2_title_dict.csv 생성"""
    
    title_url_df = pd.DataFrame(columns=['title', 'url'])
    title_url_df.to_csv('title_url.csv')
    title_url_csv = pd.read_csv('title_url.csv')

    title_list = []
    url_list = []
    pdf_list = []
    classification1 = []
    classification2 = []
    title_url_dict = {}
    cla_dict=collections.defaultdict(list)
    pdf_cla_dict={}

    for url in third_url_list: 
        driver.get(url)

        last_height = driver.execute_script("return document.body.scrollHeight") 
        while True:
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)") 
            time.sleep(3)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height       

        title_url_elem = driver.find_elements(By.CLASS_NAME, 'col-lg-9.item-content > h1 > a')
        classification2_elem=driver.find_elements(By.CLASS_NAME, 'artefact-header > h1')

        for cla2 in classification2_elem:
            classification2.append(cla2.text)            

            for title_url in title_url_elem:
                if title_url.get_attribute('href') not in title_url_csv['url']:

                    title_list.append(title_url.text)
                    url_list.append(title_url.get_attribute('href'))


                    title_url_dict[title_url.text] = title_url.get_attribute('href')
                    cla_dict[cla2.text].append(title_url.text)
                    
    title_url_df = pd.DataFrame.from_dict(title_url_dict, orient='index')
    title_url_df.to_csv('title_url.csv')
    cla_dict_df = pd.DataFrame(columns=['cla2', 'title'])
    cla_dict_df = pd.DataFrame.from_dict(cla_dict, orient='index')
    cla_dict_df.to_csv('cla2_title_dict.csv')


def get_pdf_url():
    
    """pdf주소 수집, title_cla_df.csv, title_pdf_df.csv 생성"""

    title_url_csv = pd.read_csv('title_url.csv')
    title_url_csv.rename(columns = {'Unnamed: 0':'title', '0':'url'}, inplace=True )
    url_df = title_url_csv['url']

    title_pdf_dict = collections.defaultdict(list)
    title_cla2_dict = collections.defaultdict(list)
    pdf_list = []

    for url in url_df:
        driver.get(url)

        classify_url_elem = driver.find_elements(By.CLASS_NAME, 'col-md-12 > a')
        title_elem = driver.find_elements(By.CLASS_NAME, 'col-md-12 > h1')
        pdf_url = driver.find_elements(By.CLASS_NAME,'badge.badge-light')[0].get_attribute('href')

        pdf_list.append(pdf_url)

        for classify_url in classify_url_elem:
            classify = classify_url.get_attribute('href')        

            #print(classify_url)
            for title_rough in title_elem:
                title = title_rough.text

                classify_section = classify.split('/')[-1].replace('-',' ')
                if 'paperswithcode.com/task' in classify and classify_section not in title_cla2_dict[title]:        
                    title_cla2_dict[title].append(classify_section)

                for k in pdf_url:
                    title_pdf_dict[title] = pdf_url
                    
    pdf_df = pd.DataFrame(pdf_list, columns=['pdf'])
    pdf_df.to_csv('pdf.csv')
                    
    title_cla2_df = pd.DataFrame.from_dict(title_cla2_dict, orient='index')
    title_cla2_df.to_csv('title_cla_df.csv')

    title_pdf_df = pd.DataFrame.from_dict(title_pdf_dict, orient='index')
    title_pdf_df.to_csv('title_pdf_df.csv')

def download_pdf(download_path):
    
    """pdf 다운로드"""
    
    print(f'전체 파일 수: {len(pdfs)}')

    if not os.path.exists(download_path):
        os.makedirs(download_path)
    else:
        print('directory exists')

    for num, (url, file) in enumerate(zip(pdfs, pdf_file)):

        if os.path.isfile(f'{download_path}/{file}'):
            print(f'{num} : {file}은 이미 존재하는 파일입니다.')
        else:
            print(f'{num} : {file} downloading')
            os.system(f'curl "{url}" --output "{download_path}/{file}"')

#pdf text변환: https://koreapy.tistory.com/829
def pdf_to_txt(pdf_file):
    
    rsrcmgr = PDFResourceManager()
   
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(pdf_file, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()
    
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,
                                 caching=caching, check_extractable=True):
        interpreter.process_page(page)
    
    text = retstr.getvalue()
    
    fp.close()
    device.close()
    retstr.close()
    pdf_txt=[]
    pdf_txt.append(text)
    return text

#오류제외 파일변환

def pdf_to_txt_with_exception(pdf_file, download_path):
    
    """pdf를 txt로 변환, 오류처리"""
    
    if not os.path.exists(f'{download_path}/txt'):
        os.makedirs(f'{download_path}/txt')
    else:
        print('directory exists')
        
    for pdf in tqdm(pdf_file):  
        txt_file_name = '.'.join(pdf.split('.')[:-1])
        if os.path.isfile( f'{download_path}/txt/{txt_file_name}.txt' ) != True:
            try:
                text = pdf_to_txt(f'{download_path}/{pdf}')            
                with open( f'{download_path}/txt/{txt_file_name}.txt', 'w', encoding='utf-8' ) as f:
                    f.write(text)
            except Exception as e:
                print(e, pdf)

def word_token(txt_path):
    
    """단어 토큰화, keywords.csv생성"""
    
    def get_wordnet_pos(tagged_pos):
        for pos in ['V', 'N', 'J', 'R']:
            if tagged_pos.startswith(pos):
                return pos.lower() if pos != 'J' else 'a'
        return None

    lemm = WordNetLemmatizer()
    word_txt={}

    for txt_file in tqdm(txt_path):
        with open(txt_file, encoding='utf-8') as f:
            txt = f.read().lower().replace('-\n', '')


        # 토크나이저
            re_tokenizer = RegexpTokenizer('[a-zA-Z]{2,}')

            # 토큰화
            word_tokens = re_tokenizer.tokenize(txt)

            # 불용어(stopwords) 제거
            stop_words = stopwords.words('english')
            stop_words.append('cid')
            word_tokens = [w for w in word_tokens if w not in stop_words]

            #stemmer = PorterStemmer()
            stemmer = SnowballStemmer('english')
            word_tokens = [stemmer.stem(w) for w in word_tokens]

            word_tokens = pos_tag(word_tokens)
            word_tokens = [(w, get_wordnet_pos(tag)) for w, tag in word_tokens if get_wordnet_pos(tag) != None]
            word_tokens = [lemm.lemmatize(word, pos=tag) for word, tag in word_tokens]

            if word_txt.values() == None:
                for f in FreqDist(word_tokens):
                    word_txt[f] = txt_file
                    
    freq_df = pd.DataFrame.from_dict(FreqDist(word_tokens),  orient='index')
#     freq_df.to_csv('keywords.csv')
#     freq_csv = pd.read_csv('keywords.csv')
    freq_df.rename(columns={'Unnamed: 0': 'keywords', '0': 'frequency'}, inplace=True)
    freq_df.to_csv('keywords.csv')

def google_search():
    
    """keywords 구글로 뜻 찾기"""
    
#     keywords_copy = pd.read_csv('keywords.csv')
#     keywords_copy.rename(columns={'Unnamed: 0': 'keywords', '0': 'frequency'}, inplace=True)
    keywords = keywords_copy['keywords']
    terminologies = {}
    explains = []
    
    for keyword in keywords:
        driver.get('https://www.google.com')
        search_elem = driver.find_element(By.CLASS_NAME, 'gLFyf.gsfi')
        search_elem.clear()
        search_elem.send_keys( f'what is {keyword} of deep learning' )
        driver.find_element(By.CLASS_NAME, 'gNO89b').submit()

        relatedQ_elem = driver.find_elements(By.CLASS_NAME, 'Wt5Tfe') 
        one_relatedQ_elem = driver.find_elements(By.CLASS_NAME, 'wQiwMc.ygGdYd.related-question-pair')    
        explain_snippet_elem = driver.find_elements(By.CLASS_NAME,'LGOjhe')    
        explain_check_elem = driver.find_elements(By.CLASS_NAME, 'yp1CPe.wDYxhc.NFQFxe.viOShc.LKPcQc > div > div > div > div')   
        full_keyword_elem = driver.find_elements(By.CLASS_NAME, 'gL9Hy')


        for full_keyword in full_keyword_elem:
            if keyword in full_keyword.text:
                keywords.replace(keyword, full_keyword.text.split(' ')[2], inplace=True)         

        count = 0
        try:
            for click_number, questions in enumerate(one_relatedQ_elem):
                for rQ in relatedQ_elem:
                    if rQ.is_displayed:
                        if count == 0:
                            count += 1
                            questions.click()
                            
                    for explain in explain_snippet_elem:
                        explains.append(explain.text)
                        if terminologies.get(keyword) == None:
                            terminologies[keyword] = explain.text
        except Exception as e:
            print(e)



        for explain in explain_check_elem:
            if explain.is_displayed():            
                explains.append(explain.text)
                if terminologies.get(keyword) == None:
                    time.sleep(1)
                    terminologies[keyword] = explain.text

        time.sleep(2)        
        
    word_explain = pd.DataFrame( columns=['keywords', 'explains'] )
    word_explain['keywords'] = terminologies.keys()
    word_explain['explains'] = terminologies.values()
    word_explain.to_csv('word_explains.csv')

    keywords_copy.to_csv('keywords_change.csv')

def explain_translation():
    Ttranslated = {}
    translate_result = []

    explain_translation = {}
    keywords_translation = {}
    word_explain_csv = pd.read_csv('word_explains.csv', index_col=0)

    for explain in word_explain_csv['explains']:
        translator = Translator()
        translated_explanation = translator.translate(explain, src='en', dest='ko')
        translate_result.append(translated_explanation.text)
        explain_translation[explain] = translated_explanation.text #원문-번역 dict

    explain_translation_df = pd.DataFrame( columns=['explains', 'translations'] )
    explain_translation_df['explains'] = explain_translation.keys()
    explain_translation_df['translations'] = explain_translation.values()
    explain_translation_df.to_csv('explain_translation.csv')

    keyword_explain_trans_df = word_explain_csv.merge(explain_translation_df, how='left', on='explains')
    keyword_explain_trans_frq_df = keyword_explain_trans_df.merge(keywords_df, how='left', on='keywords')
    keyword_explain_trans_frq_df = keyword_explain_trans_frq_df[['keywords','translations','frequency','explain']]
    keyword_explain_trans_frq_df.to_csv('keyword_explain_trans_frq.csv')

def order_by_frequency():


    result = {}
    sorted_result = {}
    # keywords=keywords_all['단어']
    replace_txt_name=[]

    keywords_csv = pd.read_csv('keywords.csv', index_col=0)
    keywords = keywords_copy['keywords']

    for keyword in keywords:
        result[keyword] = {}

        for txt_file in glob('./Downloads/pdf_download/txt/*.txt'):
            txt_file_name = os.path.basename(txt_file).replace('.txt','')        
            with open(txt_file, encoding='utf-8') as file:
                sentences = file.read().lower().replace('-\n', '').replace('\n', ' ').split('.')
            sentences = ' '.join(sentences)
            count = sentences.count(keyword)        

            result[keyword][txt_file_name] = count              

            """이거는 txt name으로 pdf가 몇 번 등장했는지 알려줌
            여기에다가 """





            sorting = sorted(result[keyword], key= lambda x : result[keyword][x], reverse=True)[:5]
            sorted_result[keyword] = sorting

        

warnings.filterwarnings('ignore')

options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
    "download.default_directory": "C:/Users/kimEn/anaconda3/envs/deep_learning_dictionary", #Change default directory for downloads
    "download.prompt_for_download": False, #To auto download the file
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
})
driver = webdriver.Chrome('chromedriver.exe', options=options)



class collect_informations():
    first_collect_url()
    make_classification()
    collect_title_url()
    get_pdf_url()
    
pdf_to_txt_with_exception(pdf_file, download_path)
txt_path = glob('C:/Users/kimEn/Downloads/pdf_download/txt/*.txt')
word_token(txt_path)
chrome_option()
google_search()
explain_translation()

third_url_csv = pd.read_csv('third_url.csv')
cla1_cla2_df = pd.read_csv('cla1_cla2_dict.csv', index_col=0)
cla1_cla2_df.T


pdf_csv = pd.read_csv('title_pdf_df')
pdf_csv

pdf_csv = pd.read_csv('pdf.csv', index_col=0)
pdfs = pdf_csv['pdf'].values.tolist()
pdf_file = [ pdf.split('/')[-1] for pdf in pdf_csv['pdf'].values if 'arxiv' in pdf ]
    
    
    
download_path = f'C:/Users/kimEn/Downloads/pdf_download'
download_pdf(download_path)

last_last = pd.merge(left=keywords_all, right=classification_last, how='left', on='단어')
# last_result = last_last.to_html('결과.html', encoding="utf-8-sig")

keywords_csv = pd.read_csv('keywords.csv')
keywords_csv.rename(columns={'Unnamed: 0': 'keywords', '0': 'frequency'}, inplace=True)
keywords_csv

keywords_copy = keywords_csv.copy()
keywords_copy
keywords_copy['keywords']






